In [1]:
class Cell:
    def __init__(self, x, y, Q, mapping_index):
        self.x = x
        self.y = y
        self.Q = Q
        self.mapping_index = mapping_index


    def __str__(self):
        return f'Cell({self.x}, {self.y}, {self.Q}, {self.mapping_index})'

In [2]:
ALFA = 0.595
BETA = 0.31
GAMMA = 1.58
OMEGA = 5.75

# dummies = [1,3,5,6,8,9,11,13,14,16,17,19,21,23,25,27,29,31,33,35,37]

# def has_all_dummies_between(cell1: Cell, cell2: Cell):
#   for i in range(cell1.x + 1, cell2.x):
#     if i not in dummies:
#       return False
#   return True


def vertical_distance_in_same_block(cell1: Cell, cell2: Cell, s: int, path_info: list):
    # if cell1.Q == 2 and cell2.Q == 2 and has_all_dummies_between(cell1, cell2):
    #   return BETA*abs(cell1.y - cell2.y)

    if cell1.x == cell2.x:
      path_info.append(0)
      return BETA*abs(cell1.y - cell2.y)
        
    else:
      if (cell1.y + cell2.y) <= s:
        if 2*s - cell1.y - cell2.y < cell1.y + cell2.y:
          path_info.append(1)
          return BETA*(2*s - cell1.y - cell2.y);
        else:
          path_info.append(2)
          return BETA*(cell1.y + cell2.y - 2)
      else:
        path_info.append(1)
        return BETA*((s-cell1.y) + (s-cell2.y) + 2) # Yukardan gittiğim garanti?



def horizontal_distance(cell1: Cell, cell2: Cell):
    return abs(cell1.x - cell2.x)*(2*ALFA + GAMMA)


def calculate_distance_in_same_block(cell1: Cell, cell2: Cell, s: int, path_info: list):
    return vertical_distance_in_same_block(cell1, cell2, s, path_info) + horizontal_distance(cell1, cell2)

In [3]:
import openpyxl
import pandas as pd
from scipy.spatial import distance_matrix
import json

# Define variable to load the dataframe
dataframe = openpyxl.load_workbook("./mezanin_cells_by_floors.xlsx")

# Define variable to read sheet
sheets = dataframe.sheetnames

In [4]:
sheets = [sheets[0]]

In [5]:
data_by_floor = []

for sheet in sheets:
  data = {}
  dataframe1 = dataframe[sheet]
  cells = []
  last_cell = Cell(0,0,0,0)
  mapping_index = 0
  y = 1
  x = 1
  Q = 1
  mapping_dic = {}

  for col_index in range(0, dataframe1.max_column):
    mapping_index = mapping_index + 1
    cells.append(Cell(x, y, Q, mapping_index))
    for row_index in range(0, dataframe1.max_row):

      code = dataframe1.cell(row_index + 1, col_index + 1).value

      if (row_index - 5) % 10 == 0:
        mapping_index = mapping_index + 1
        x += 1

      if (row_index - 5) % 10 == 0:
        new_cell = Cell(x, y, Q, mapping_index)
        cells.append(new_cell)

      if code is not None:
        mapping_dic[code] = mapping_index
        #dataframe1.cell(row_index + 1, col_index + 1).value = mapping_index



    x = 1
    y = y + 1

  data['cells'] = cells
  data['mapping_dic'] = mapping_dic

  data_by_floor.append(data)


In [6]:
len(data_by_floor[0]['cells'])

3861

In [7]:
# with pd.ExcelWriter("./cells_with_mapping_index.xlsx") as writer:

#     for sheet_name in sheets:
#       df = pd.DataFrame(dataframe[sheet_name].values)
#       # use to_excel function and specify the sheet_name and index
#       # to store the dataframe in specified sheet
#       df.to_excel(writer, sheet_name=sheet_name, index=True)

In [8]:
floor_dataframes = []
floor_path_df = []

for i, floor_data in enumerate(data_by_floor):

  with open(f"{sheets[i]}.json",'w') as fi:
    json.dump(floor_data['mapping_dic'],fi)

  cells = floor_data['cells']
  distance_data = []
  mapping_indexes = []
  path_info = []

  for i in range(len(cells)):
      mapping_indexes.append(cells[i].mapping_index)
      tmp_dist_data = []
      tmp_path_info = []
      for j in range(len(cells)):
          if i != j:
              tmp_dist_data.append(calculate_distance_in_same_block(cells[i], cells[j], 99, tmp_path_info))
          else:
              tmp_path_info.append(0)
              tmp_dist_data.append(0)
      distance_data.append(tmp_dist_data)
      path_info.append(tmp_path_info)

  df = pd.DataFrame(distance_data, columns=mapping_indexes, index=mapping_indexes)
  path_df = pd.DataFrame(path_info, columns=mapping_indexes, index=mapping_indexes)

  floor_dataframes.append(df)
  floor_path_df.append(path_df)

In [9]:
floor_dataframes[0][2482][2378]

59.26

In [10]:
with pd.ExcelWriter("./distance_matrix_by_floor.xlsx") as writer:

    for i, df in enumerate(floor_dataframes):
      # use to_excel function and specify the sheet_name and index
      # to store the dataframe in specified sheet
      df.to_excel(writer, sheet_name=sheets[i], index=True)

In [49]:
# with pd.ExcelWriter("./shortest_path_matrix_by_floor.xlsx") as writer:

#     for i, df in enumerate(floor_path_df):
#       # use to_excel function and specify the sheet_name and index
#       # to store the dataframe in specified sheet
#       df.to_excel(writer, sheet_name=sheets[i], index=True)

In [ ]:
# distance_data = []
# mapping_indexes = []
# for i in range(len(cells)):
#     mapping_indexes.append(cells[i].mapping_index)
#     tmp_dist_data = []
#     for j in range(len(cells)):
#         if i != j:
#             tmp_dist_data.append(calculate_distance_in_same_block(cells[i], cells[j], 99))
#         else:
#             tmp_dist_data.append(0)
#     distance_data.append(tmp_dist_data)
#     print(round(i / len(cells) * 100), '%')

# df = pd.DataFrame(distance_data, columns=mapping_indexes, index=mapping_indexes)

# df.to_excel("distance_matrix.xlsx", index=True)

In [ ]:
# print(dataframe1.max_column)
# cells = []

# last_cell = Cell(0,0,0,0)

# mapping_index = 0

# y = 1
# x = 0
# Q = 1

# mapping_dic = {}

# for col_index in range(0, dataframe1.max_column):
#   for row_index in range(0, dataframe1.max_row):

#     code = dataframe1.cell(row_index + 1, col_index + 1).value

#     if code is None:
#       continue

#     if row_index % 10 == 0:
#       mapping_index = mapping_index + 1
#       x += 1
#       new_cell = Cell(x, y, Q, mapping_index)
#       last_cell = new_cell
#       cells.append(last_cell)

#     mapping_dic[code] = mapping_index

#   if col_index == 100:
#     Q = Q + 1
#     y = 0

#   x = 0
#   y = y + 1
